# Tarea 2: Gráficos estadísticos interactivos con plotly

# Introducción
Costa Rica es conocida por su rica biodiversidad, alberga varias especies de felinos que desempeñan un papel esencial en el equilibrio ecológico. Desde el jaguar hasta el ocelote, estos felinos se encuentran en diferentes ecosistemas del país. Sin embargo, enfrentan desafíos significativos debido a la pérdida de hábitat, la caza furtiva y otros factores antropogénicos.

Este sitio web tiene como objetivo proporcionar una visión de los registros de observaciones de tres especies de felinos en Costa Rica a lo largo del período de 2014 a 2024. Estas observaciones permiten comprender mejor la distribución y la frecuencia de avistamientos. La serie de gráficos a continuación ilustra la cantidad de observaciones por año, la distribución mensual en un año específico y la proporción relativa de avistamientos entre las especies.

### Especies a tratar:
##### ___Tabla #1___

|   Nombre de  la Especie   | Nombre cientifico    | 
|:-------------------------:|:--------------------:|
| Jaguar                    |  (_Panthera onca_)   |
| Ocelote                   |(_Leopardus pardalis_)|
| Puma                      |  (_Puma concolor_)   |


# Carga de paquetes

In [37]:
# Carga de plotly.express con el alias px
import plotly.express as px

# Carga de plotly.graph_objects con el alias go
import plotly.graph_objects as go

# Carga de requests y de json
import requests
import json

# Carga de pandas y geopandas
import pandas as pd
import geopandas as gpd


# Obtención de Datos

## Jaguar *(Panthera onca)*

In [3]:
import requests
import json

def obtener_taxon_key(nombre_cientifico):
    url = 'https://api.gbif.org/v1/species/match'

    parametros = {
        'name': nombre_cientifico,
        'strict': True  # Asegura una coincidencia exacta
    }

    respuesta = requests.get(url, params=parametros)
    
    if respuesta.status_code == 200:
        datos = respuesta.json()
        
        # Se revisa si la llave usageKy está presente en datos
        if 'usageKey' in datos:
            return datos['usageKey']
        else:
            return None
    else:
        return None

# Nombre científico de la especie
nombre_cientifico = 'Panthera onca'

# taxonKey de la especie
taxon_key = obtener_taxon_key(nombre_cientifico)

if taxon_key:
    # Se encontró el taxonKey

    # URL
    url = 'https://api.gbif.org/v1/occurrence/search'

    # Parámetros de la solicitud GET
    parametros = {
        'country': 'CR',          # Código de país ISO para Costa Rica
        'taxonKey': taxon_key,
        'limit': 300,             # Número máximo de registros a obtener (máximo 300 por solicitud)
        'hasCoordinate': 'true',   # Solo se incluyen registros de presencia con coordenadas
        'year': '2014,2024'  # Rango de años especificado
    }

    # Solicitud GET
    respuesta = requests.get(url, params=parametros)

    # Se verifica si la solicitud fue exitosa
    if respuesta.status_code == 200:
        # La solicitud fue exitosa

        # Se cargan en formato JSON los datos retornados
        datos_registros = respuesta.json()

        # Se extraen las ocurrencias
        registros = datos_registros['results']

        # Lista para las features de GeoJSON
        features = []

        # Se recorren los datos de registros de presencia y se convierten a features GeoJSON
        for registro in registros:
            especie = registro.get('species')
            fecha = registro.get('eventDate')
            localidad = registro.get('locality')
            latitud = registro.get('decimalLatitude')
            longitud = registro.get('decimalLongitude')

            print(f"Especie: {especie}")
            print(f"Fecha: {fecha}")
            print(f"Localidad: {localidad}")
            print(f"Longitud: {longitud}, Latitud: {latitud}")
            print("-" * 40)        

            # Se verifica que las coordenadas están presentes
            if latitud is not None and longitud is not None:
                feature = {
                    "type": "Feature",
                    "geometry": {
                        "type": "Point",
                        "coordinates": [longitud, latitud]  # GeoJSON utiliza el orden [longitud, latitud]
                    },
                    "properties": {
                        "species": especie,
                        "eventDate": fecha,
                        "locality": localidad
                    }
                }
                features.append(feature)

 # Crear el FeatureCollection GeoJSON
        geojson = {
            "type": "FeatureCollection",
            "features": features
        }

        # Almacenamiento de datos en un archivo llamado "datos-registros-presencia.geojson"
        with open('registros-presencia-jaguar.geojson', 'w', encoding='utf-8') as archivo:
            json.dump(geojson, archivo, indent=4, ensure_ascii=False)

        print(f"Se han guardado {len(features)} registros de presencia en el archivo 'registros-presencia-jaguar.geojson'")
    else:
        # Se produjo un error
        print(f"Error {respuesta.status_code}")
else:
    print(f'No se encontró el taxonKey de {nombre_cientifico}')      

Especie: Panthera onca
Fecha: 2024-01-30T15:18:56
Localidad: None
Longitud: -83.579157, Latitud: 10.467856
----------------------------------------
Especie: Panthera onca
Fecha: 2024-01-25T07:16
Localidad: None
Longitud: -84.655167, Latitud: 10.22839
----------------------------------------
Especie: Panthera onca
Fecha: 2024-02-01T13:24:14
Localidad: None
Longitud: -83.566878, Latitud: 10.449497
----------------------------------------
Especie: Panthera onca
Fecha: 2024-02-06T09:51
Localidad: None
Longitud: -83.565366, Latitud: 8.466594
----------------------------------------
Especie: Panthera onca
Fecha: 2024-02-08T16:38
Localidad: None
Longitud: -83.563344, Latitud: 10.454353
----------------------------------------
Especie: Panthera onca
Fecha: 2024-02-09T11:42
Localidad: None
Longitud: -83.548497, Latitud: 10.469183
----------------------------------------
Especie: Panthera onca
Fecha: 2024-02-09T11:45
Localidad: None
Longitud: -83.449484, Latitud: 10.556402
----------------------

## Ocelote *(Leopardus pardalis)*

In [4]:
import requests
import json

def obtener_taxon_key(nombre_cientifico):
    url = 'https://api.gbif.org/v1/species/match'

    parametros = {
        'name': nombre_cientifico,
        'strict': True  # Asegura una coincidencia exacta
    }

    respuesta = requests.get(url, params=parametros)
    
    if respuesta.status_code == 200:
        datos = respuesta.json()
        
        # Se revisa si la llave usageKy está presente en datos
        if 'usageKey' in datos:
            return datos['usageKey']
        else:
            return None
    else:
        return None

# Nombre científico de la especie
nombre_cientifico = 'Leopardus pardalis'

# taxonKey de la especie
taxon_key = obtener_taxon_key(nombre_cientifico)

if taxon_key:
    # Se encontró el taxonKey

    # URL
    url = 'https://api.gbif.org/v1/occurrence/search'

    # Parámetros de la solicitud GET
    parametros = {
        'country': 'CR',          # Código de país ISO para Costa Rica
        'taxonKey': taxon_key,
        'limit': 300,             # Número máximo de registros a obtener (máximo 300 por solicitud)
        'hasCoordinate': 'true',   # Solo se incluyen registros de presencia con coordenadas
        'year': '2014,2024'  # Rango de años especificado
    }

    # Solicitud GET
    respuesta = requests.get(url, params=parametros)

    # Se verifica si la solicitud fue exitosa
    if respuesta.status_code == 200:
        # La solicitud fue exitosa

        # Se cargan en formato JSON los datos retornados
        datos_registros = respuesta.json()

        # Se extraen las ocurrencias
        registros = datos_registros['results']

        # Lista para las features de GeoJSON
        features = []

        # Se recorren los datos de registros de presencia y se convierten a features GeoJSON
        for registro in registros:
            especie = registro.get('species')
            fecha = registro.get('eventDate')
            localidad = registro.get('locality')
            latitud = registro.get('decimalLatitude')
            longitud = registro.get('decimalLongitude')

            print(f"Especie: {especie}")
            print(f"Fecha: {fecha}")
            print(f"Localidad: {localidad}")
            print(f"Longitud: {longitud}, Latitud: {latitud}")
            print("-" * 40)        

            # Se verifica que las coordenadas están presentes
            if latitud is not None and longitud is not None:
                feature = {
                    "type": "Feature",
                    "geometry": {
                        "type": "Point",
                        "coordinates": [longitud, latitud]  # GeoJSON utiliza el orden [longitud, latitud]
                    },
                    "properties": {
                        "species": especie,
                        "eventDate": fecha,
                        "locality": localidad
                    }
                }
                features.append(feature)

# Crear el FeatureCollection GeoJSON
        geojson = {
            "type": "FeatureCollection",
            "features": features
        }

        # Almacenamiento de datos en un archivo llamado "datos-registros-presencia.geojson"
        with open('registros-presencia-ocelote.geojson', 'w', encoding='utf-8') as archivo:
            json.dump(geojson, archivo, indent=4, ensure_ascii=False)

        print(f"Se han guardado {len(features)} registros de presencia en el archivo 'registros-presencia-ocelote.geojson'")
    else:
        # Se produjo un error
        print(f"Error {respuesta.status_code}")
else:
    print(f'No se encontró el taxonKey de {nombre_cientifico}')

Especie: Leopardus pardalis
Fecha: 2024-01-08T20:12
Localidad: None
Longitud: -83.358795, Latitud: 8.76312
----------------------------------------
Especie: Leopardus pardalis
Fecha: 2024-02-15T21:52
Localidad: None
Longitud: -85.072136, Latitud: 9.868216
----------------------------------------
Especie: Leopardus pardalis
Fecha: 2024-02-15T08:29:13
Localidad: None
Longitud: -84.792229, Latitud: 10.357254
----------------------------------------
Especie: Leopardus pardalis
Fecha: 2024-02-15T21:52
Localidad: None
Longitud: -85.072136, Latitud: 9.868216
----------------------------------------
Especie: Leopardus pardalis
Fecha: 2024-02-17T20:57:55
Localidad: None
Longitud: -83.417847, Latitud: 8.429842
----------------------------------------
Especie: Leopardus pardalis
Fecha: 2024-02-29T09:58:19
Localidad: None
Longitud: -83.068289, Latitud: 9.813608
----------------------------------------
Especie: Leopardus pardalis
Fecha: 2024-03-03T08:04:58
Localidad: None
Longitud: -85.334288, Lati

## Puma *(Puma concolor)*

In [5]:
import requests
import json

def obtener_taxon_key(nombre_cientifico):
    url = 'https://api.gbif.org/v1/species/match'

    parametros = {
        'name': nombre_cientifico,
        'strict': True  # Asegura una coincidencia exacta
    }

    respuesta = requests.get(url, params=parametros)
    
    if respuesta.status_code == 200:
        datos = respuesta.json()
        
        # Se revisa si la llave usageKy está presente en datos
        if 'usageKey' in datos:
            return datos['usageKey']
        else:
            return None
    else:
        return None

# Nombre científico de la especie
nombre_cientifico = 'Puma concolor'

# taxonKey de la especie
taxon_key = obtener_taxon_key(nombre_cientifico)

if taxon_key:
    # Se encontró el taxonKey

    # URL
    url = 'https://api.gbif.org/v1/occurrence/search'

    # Parámetros de la solicitud GET
    parametros = {
        'country': 'CR',          # Código de país ISO para Costa Rica
        'taxonKey': taxon_key,
        'limit': 300,             # Número máximo de registros a obtener (máximo 300 por solicitud)
        'hasCoordinate': 'true',   # Solo se incluyen registros de presencia con coordenadas
        'year': '2014,2024'  # Rango de años especificado
    }

    # Solicitud GET
    respuesta = requests.get(url, params=parametros)

    # Se verifica si la solicitud fue exitosa
    if respuesta.status_code == 200:
        # La solicitud fue exitosa

        # Se cargan en formato JSON los datos retornados
        datos_registros = respuesta.json()

        # Se extraen las ocurrencias
        registros = datos_registros['results']

        # Lista para las features de GeoJSON
        features = []

        # Se recorren los datos de registros de presencia y se convierten a features GeoJSON
        for registro in registros:
            especie = registro.get('species')
            fecha = registro.get('eventDate')
            localidad = registro.get('locality')
            latitud = registro.get('decimalLatitude')
            longitud = registro.get('decimalLongitude')

            print(f"Especie: {especie}")
            print(f"Fecha: {fecha}")
            print(f"Localidad: {localidad}")
            print(f"Longitud: {longitud}, Latitud: {latitud}")
            print("-" * 40)        

            # Se verifica que las coordenadas están presentes
            if latitud is not None and longitud is not None:
                feature = {
                    "type": "Feature",
                    "geometry": {
                        "type": "Point",
                        "coordinates": [longitud, latitud]  # GeoJSON utiliza el orden [longitud, latitud]
                    },
                    "properties": {
                        "species": especie,
                        "eventDate": fecha,
                        "locality": localidad
                    }
                }
                features.append(feature)

# Crear el FeatureCollection GeoJSON
        geojson = {
            "type": "FeatureCollection",
            "features": features
        }

        # Almacenamiento de datos en un archivo llamado "datos-registros-presencia.geojson"
        with open('registros-presencia-puma.geojson', 'w', encoding='utf-8') as archivo:
            json.dump(geojson, archivo, indent=4, ensure_ascii=False)

        print(f"Se han guardado {len(features)} registros de presencia en el archivo 'registros-presencia-puma.geojson'")
    else:
        # Se produjo un error
        print(f"Error {respuesta.status_code}")
else:
    print(f'No se encontró el taxonKey de {nombre_cientifico}')

Especie: Puma concolor
Fecha: 2024-01-06T12:34:16
Localidad: None
Longitud: -85.107174, Latitud: 9.567627
----------------------------------------
Especie: Puma concolor
Fecha: 2024-01-28T14:51
Localidad: None
Longitud: -85.092885, Latitud: 9.587534
----------------------------------------
Especie: Puma concolor
Fecha: 2024-01-30T11:47
Localidad: None
Longitud: -85.092885, Latitud: 9.587534
----------------------------------------
Especie: Puma concolor
Fecha: 2024-02-23T12:11:01
Localidad: None
Longitud: -84.010025, Latitud: 10.436928
----------------------------------------
Especie: Puma concolor
Fecha: 2024-02-29T10:23:18
Localidad: None
Longitud: -83.068877, Latitud: 9.811892
----------------------------------------
Especie: Puma concolor
Fecha: 2024-02-20T12:22:26
Localidad: None
Longitud: -83.732378, Latitud: 8.639435
----------------------------------------
Especie: Puma concolor
Fecha: 2024-03-20T07:26
Localidad: None
Longitud: -84.470609, Latitud: 10.028272
-------------------

# Gráficos

## Gráfico de líneas

El gráfico de líneas revela cómo han fluctuado las observaciones de jaguares, ocelotes y pumas entre 2014 y 2024. Muestra diferencias anuales claras, destacando en qué momentos una especie fue observada con mayor frecuencia. Las líneas están coloreadas distintivamente: verde para el jaguar, azul para el ocelote y rojo para el puma. Este formato permite captar visualmente patrones de aumento o disminución en los registros de cada especie, algo que sería menos evidente en una tabla o un mapa. Además, los picos específicos de observación sugieren posibles eventos o cambios en los esfuerzos de monitoreo o en la ecología de las especies, facilitando así una interpretación rápida de tendencias y comparaciones simultáneas.

In [38]:
import pandas as pd
import geopandas as gpd
import plotly.express as px

# Lectura de cada archivo GeoJSON en un GeoDataFrame con la columna de especie
gdf_jaguar = gpd.read_file('registros-presencia-jaguar.geojson')
gdf_jaguar['especie'] = 'Jaguar'

gdf_ocelote = gpd.read_file('registros-presencia-ocelote.geojson')
gdf_ocelote['especie'] = 'Ocelote'

gdf_puma = gpd.read_file('registros-presencia-puma.geojson')
gdf_puma['especie'] = 'Puma'

# Concatenación de todos los registros en un solo DataFrame
gdf = pd.concat([gdf_jaguar, gdf_ocelote, gdf_puma], ignore_index=True)

# Conversión de 'eventDate' a hilera de texto y luego a formato datetime
gdf['eventDate'] = gdf['eventDate'].astype(str)
gdf['eventDate'] = pd.to_datetime(gdf['eventDate'], errors='coerce')

# Extracción del año
gdf['anio'] = gdf['eventDate'].dt.year

# Filtrar el rango de años de interés (2014-2024)
gdf = gdf[(gdf['anio'] >= 2014) & (gdf['anio'] <= 2024)]

# Conteo de la cantidad de registros de presencia por año y especie
registros_por_anio_especie = gdf.groupby(['anio', 'especie']).size().reset_index(name='cantidad')

# Crear un DataFrame con todos los años en el rango 2014-2024 para cada especie
todos_los_anios_especies = pd.DataFrame({
    'anio': list(range(2014, 2025)) * 3,
    'especie': ['Jaguar'] * 11 + ['Ocelote'] * 11 + ['Puma'] * 11
})

# Unir ambos DataFrames para asegurarse de que todos los años estén presentes para cada especie
registros_por_anio_especie = todos_los_anios_especies.merge(
    registros_por_anio_especie, 
    on=['anio', 'especie'], 
    how='left'
)
registros_por_anio_especie['cantidad'] = registros_por_anio_especie['cantidad'].fillna(0)  # Rellenar NaN con 0

# Crear un gráfico de líneas interactivo con plotly
fig = px.line(
    registros_por_anio_especie,
    x='anio',
    y='cantidad',
    color='especie',
    title='Cantidad de registros de presencia por año (2014-2024) para jaguares, ocelotes y pumas',
    labels={'anio': 'Año', 'cantidad': 'Cantidad de registros de presencia', 'especie': 'Especie'},
     color_discrete_map={
        'Jaguar': 'green',
        'Ocelote': 'blue',
        'Puma': 'red'
    }  # Colores específicos para cada especie
)

# Ajustes al gráfico
fig.update_layout(
    xaxis_title='Año',
    yaxis_title='Registros de presencia',
    title_x=0.5,
    template='plotly_white'
)

# Despliegue del gráfico
fig.show()




Además, en base a las preguntas de la Tarea 1. Con este gráfico de líneas se podría dar una respuesta general a las siguientes preguntas:

**¿Existen tendencias que indiquen una expansión o contracción de sus hábitats?**

Se puede pensar en un cambio en en la distribución de las especies, pero hacen falta datos espaciales, para confirmar una expansión o contracción en un área específica.

**¿Existen regiones donde los esfuerzos de conservación deberían priorizarse según la presencia de especies vulnerables?**

Un aumento en registros permite considerar el señalamiento de áreas de interés para conservación.

## Gráficos de barras

### Jaguar - 2019

Este gráfico detalla el número de observaciones mensuales de jaguares durante el año 2019. Cada mes está representado, incluyendo aquellos en los que no se registraron observaciones, lo cual ayuda a analizar la variabilidad de los registros a lo largo de las estaciones. Con un total anual indicado, es posible observar patrones mensuales que pueden correlacionarse con factores climáticos o comportamentales de la especie (Arroyo *et al*, 2014).

In [33]:
import pandas as pd
import geopandas as gpd
import plotly.express as px

# Lista de nombres de meses en español
meses_espanol = [
    'Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
    'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre'
]

# Lectura del archivo GeoJSON específico de jaguares en un GeoDataFrame
gdf_jaguar = gpd.read_file('registros-presencia-jaguar.geojson')

# Conversión de 'eventDate' a hilera de texto y luego a formato datetime
gdf_jaguar['eventDate'] = gdf_jaguar['eventDate'].astype(str)
gdf_jaguar['eventDate'] = pd.to_datetime(gdf_jaguar['eventDate'], errors='coerce')

# Filtrar los datos para el año 2019
gdf_jaguar_2019 = gdf_jaguar[gdf_jaguar['eventDate'].dt.year == 2019]

# Extracción del mes
gdf_jaguar_2019['mes'] = gdf_jaguar_2019['eventDate'].dt.month

# Conteo de la cantidad de registros de presencia por mes en 2019
registros_por_mes_2019 = gdf_jaguar_2019.groupby('mes').size().reset_index(name='cantidad')

# Crear un DataFrame con todos los meses del año
todos_los_meses = pd.DataFrame({'mes': range(1, 13)})

# Unir ambos DataFrames para asegurarse de que todos los meses estén presentes
registros_por_mes_2019 = todos_los_meses.merge(registros_por_mes_2019, on='mes', how='left')
registros_por_mes_2019['cantidad'] = registros_por_mes_2019['cantidad'].fillna(0)  # Rellenar NaN con 0

# Agregar la columna con los nombres de los meses en español
registros_por_mes_2019['mes_nombre'] = registros_por_mes_2019['mes'].apply(lambda x: meses_espanol[x - 1])

# Crear un gráfico de barras para mostrar la cantidad de observaciones por mes en 2019
fig = px.bar(
    registros_por_mes_2019,
    x='mes_nombre',
    y='cantidad',
    title='Cantidad de registros de presencia de jaguares por mes en 2019 (Total: 36)',
    labels={'mes_nombre': 'Mes', 'cantidad': 'Cantidad de registros de presencia'},
    text='cantidad',
    color_discrete_sequence=['green']  # Configurar el color de las barras en verde
)

# Ajustes al gráfico
fig.update_layout(
    xaxis_title='Mes',
    yaxis_title='Cantidad de registros de presencia',
    title_x=0.5,
    template='plotly_white'
)

# Despliegue del gráfico
fig.show()


c:\Users\mfall\miniconda3\envs\geopython\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Ocelotes - 2023

Este gráfico de barras muestra el número de observaciones mensuales de ocelotes en el año 2023. El gráfico tiene las barras en azul y cuenta con el total de 130 observaciones.

El gráfico presenta la cantidad de registros de presencia de ocelotes por mes, con los nombres de los meses en español en el eje x y la cantidad de registros en el eje y. Esta visualización permite observar patrones de actividad mensual, como posibles aumentos o disminuciones en observaciones en función de la temporada.

In [32]:
import pandas as pd
import geopandas as gpd
import plotly.express as px

# Lista de nombres de meses en español
meses_espanol = [
    'Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
    'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre'
]

# Lectura del archivo GeoJSON específico de ocelotes en un GeoDataFrame
gdf_ocelote = gpd.read_file('registros-presencia-ocelote.geojson')

# Conversión de 'eventDate' a hilera de texto y luego a formato datetime
gdf_ocelote['eventDate'] = gdf_ocelote['eventDate'].astype(str)
gdf_ocelote['eventDate'] = pd.to_datetime(gdf_ocelote['eventDate'], errors='coerce')

# Filtrar los datos para el año 2023
gdf_ocelote_2023 = gdf_ocelote[gdf_ocelote['eventDate'].dt.year == 2023]

# Extracción del mes
gdf_ocelote_2023['mes'] = gdf_ocelote_2023['eventDate'].dt.month

# Conteo de la cantidad de registros de presencia por mes en 2023
registros_por_mes_2023 = gdf_ocelote_2023.groupby('mes').size().reset_index(name='cantidad')

# Crear un DataFrame con todos los meses del año
todos_los_meses = pd.DataFrame({'mes': range(1, 13)})

# Unir ambos DataFrames para asegurarse de que todos los meses estén presentes
registros_por_mes_2023 = todos_los_meses.merge(registros_por_mes_2023, on='mes', how='left')
registros_por_mes_2023['cantidad'] = registros_por_mes_2023['cantidad'].fillna(0)  # Rellenar NaN con 0

# Agregar la columna con los nombres de los meses en español
registros_por_mes_2023['mes_nombre'] = registros_por_mes_2023['mes'].apply(lambda x: meses_espanol[x - 1])

# Crear un gráfico de barras para mostrar la cantidad de observaciones por mes en 2023
fig = px.bar(
    registros_por_mes_2023,
    x='mes_nombre',
    y='cantidad',
    title='Cantidad de registros de presencia de ocelotes por mes en 2023 (Total: 130)',
    labels={'mes_nombre': 'Mes', 'cantidad': 'Cantidad de registros de presencia'},
    text='cantidad',
    color_discrete_sequence=['blue']  # Configurar el color de las barras en azul
)

# Ajustes al gráfico
fig.update_layout(
    xaxis_title='Mes',
    yaxis_title='Cantidad de registros de presencia',
    title_x=0.5,
    template='plotly_white'
)

# Despliegue del gráfico
fig.show()


c:\Users\mfall\miniconda3\envs\geopython\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Pumas - 2019

El siguiente gráfico de barras muestra el número de observaciones mensuales de pumas en el año 2019. Las barras están coloreadas de rojo y se incluye el total de 59 observaciones.

Al igual que los dos gráficos de barras anteriores, este presenta la cantidad de registros de presencia de ocelotes por mes, con los nombres de los meses en español en el eje x y la cantidad de registros en el eje y.
Y es posible observar patrones mensuales que pueden correlacionarse con factores climáticos o comportamentales de la especie.

In [31]:
import pandas as pd
import geopandas as gpd
import plotly.express as px

# Lista de nombres de meses en español
meses_espanol = [
    'Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
    'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre'
]

# Lectura del archivo GeoJSON específico de pumas en un GeoDataFrame
gdf_puma = gpd.read_file('registros-presencia-puma.geojson')

# Conversión de 'eventDate' a hilera de texto y luego a formato datetime
gdf_puma['eventDate'] = gdf_puma['eventDate'].astype(str)
gdf_puma['eventDate'] = pd.to_datetime(gdf_puma['eventDate'], errors='coerce')

# Filtrar los datos para el año 2019p
gdf_puma_2019p = gdf_puma[gdf_puma['eventDate'].dt.year == 2019]

# Extracción del mes
gdf_puma_2019p['mes'] = gdf_puma_2019p['eventDate'].dt.month

# Conteo de la cantidad de registros de presencia por mes en 2019p
registros_por_mes_2019p = gdf_puma_2019p.groupby('mes').size().reset_index(name='cantidad')

# Crear un DataFrame con todos los meses del año
todos_los_meses = pd.DataFrame({'mes': range(1, 13)})

# Unir ambos DataFrames para asegurarse de que todos los meses estén presentes
registros_por_mes_2019p = todos_los_meses.merge(registros_por_mes_2019p, on='mes', how='left')
registros_por_mes_2019p['cantidad'] = registros_por_mes_2019p['cantidad'].fillna(0)  # Rellenar NaN con 0

# Agregar la columna con los nombres de los meses en español
registros_por_mes_2019p['mes_nombre'] = registros_por_mes_2019p['mes'].apply(lambda x: meses_espanol[x - 1])

# Crear un gráfico de barras para mostrar la cantidad de observaciones por mes en 2019p
fig = px.bar(
    registros_por_mes_2019p,
    x='mes_nombre',
    y='cantidad',
    title='Cantidad de registros de presencia de pumas por mes en 2019 (Total: 59)',
    labels={'mes_nombre': 'Mes', 'cantidad': 'Cantidad de registros de presencia'},
    text='cantidad',
    color_discrete_sequence=['red']  # Configurar el color de las barras en rojo
)

# Ajustes al gráfico
fig.update_layout(
    xaxis_title='Mes',
    yaxis_title='Cantidad de registros de presencia',
    title_x=0.5,
    template='plotly_white'
)

# Despliegue del gráfico
fig.show()


c:\Users\mfall\miniconda3\envs\geopython\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Los gráficos presentan un patrón de aumento en los registros de jaguares, ocelotes y pumas entre 2014 y 2024, lo que sugiere una mejora en la visibilidad de estas especies o en los esfuerzos de conservación realizados. Además, las variaciones mensuales observadas en los gráficos de ocelotes y pumas indican diferencias estacionales en las observaciones, posiblemente influenciadas por factores ambientales como la disponibilidad de alimento o la actividad humana (Montalvo *et al*, 2015).

Lo interesante de estos gráficos es que permiten identificar rápidamente tendencias y patrones que podrían pasar desapercibidos en formatos más tradicionales. Por ejemplo, se puede apreciar de inmediato cuáles son los meses con más avistamientos y cómo varían las actividades de cada especie a lo largo del tiempo. Esta visualización clara ayuda a dirigir los esfuerzos de conservación hacia momentos y lugares específicos donde la necesidad es más urgente.

## Gráfico pastel

El gráfico de pastel ilustra la distribución de las observaciones de tres especies de felinos: jaguares, ocelotes y pumas, entre los años 2014 y 2024. Cada sección del gráfico representa la proporción de registros para cada especie, lo que permite visualizar fácilmente cuál de ellas fue observada con mayor frecuencia durante este período.

In [34]:
import pandas as pd
import geopandas as gpd
import plotly.express as px

# Lectura de cada archivo GeoJSON en un GeoDataFrame con la columna de especie
gdf_jaguar = gpd.read_file('registros-presencia-jaguar.geojson')
gdf_jaguar['especie'] = 'Jaguar'

gdf_ocelote = gpd.read_file('registros-presencia-ocelote.geojson')
gdf_ocelote['especie'] = 'Ocelote'

gdf_puma = gpd.read_file('registros-presencia-puma.geojson')
gdf_puma['especie'] = 'Puma'

# Concatenar todos los registros en un solo DataFrame
df = pd.concat([gdf_jaguar, gdf_ocelote, gdf_puma], ignore_index=True)

# Filtrar el rango de años (2014-2024) en la columna de fecha
df['eventDate'] = pd.to_datetime(df['eventDate'], errors='coerce')
df['year'] = df['eventDate'].dt.year
df = df[(df['year'] >= 2014) & (df['year'] <= 2024)]

# Contar la cantidad de registros por especie
conteo_especies = df['especie'].value_counts().reset_index()
conteo_especies.columns = ['especie', 'cantidad']

# Crear el gráfico de pastel con Plotly Express
fig = px.pie(
    conteo_especies,
    names='especie',
    values='cantidad',
    title='Distribución de observaciones de jaguares, ocelotes y pumas (2014-2024)'
)

# Ajustar los colores manualmente en los trazos
fig.update_traces(marker=dict(colors=['blue', 'red', 'green']))

# Mostrar el gráfico
fig.show()


Asimismo el gráfico revela de manera clara cuál de las tres especies fue más observada durante el periodo analizado. Esto permite identificar si existe una especie dominante en términos de avistamientos, lo que podría indicar su mayor presencia en el hábitat, así como su adaptabilidad o respuesta a cambios ambientales o humanos (Montalvo *et al*, 2015).

En este sentido, el uso de un gráfico de pastel facilita la apreciación de la relación proporcional entre las especies, algo que podría no ser evidente en una tabla de datos. Al observar visualmente la relación entre los registros, se pueden identificar patrones como el predominio de una especie sobre las otras de manera rápida y efectiva, lo que no siempre es tan evidente en formatos de texto o mapas.

## Referencias:

Arroyo-Arce, S., Guilder, J., & Salom-Pérez, R. (2014). Características del hábitat que influyen en la presencia de jaguar (Panthera onca) (Carnivora: Felidae) en el Parque Nacional Tortuguero, Costa Rica. Revista de Biología Tropical, 62(4), 1411-1422. https://doi.org/10.15517/rbt.v62i4.10807

Montalvo Guadamuz, V., Sáenz Bolaños, C., Ramírez Carvajal, S., & Carrillo Jiménez, E. (2015). Abundancia del jaguar (Panthera onca), otros felinos y sus presas potenciales en el Parque Nacional Santa Rosa, Costa Rica. Cuadernos de Investigación UNED, 7(2), 1-15. https://doi.org/10.22458/ciu.v7i2.1420